# Snowflake AI Stack: Notebook

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from src.observability import start_observability

session = start_observability()

session.reset_database()

## Create and Load Vector Store

In [ ]:
from src.retrieval import VectorStore

vector_store = VectorStore()

docs = vector_store.load_text_files(file_path="./data.txt")

chunks = vector_store.split_documents(documents = docs)

vector_store.add_chunks(chunks)

## Set LLM

In [ ]:
from src.generation import ChatModel

chat_model = ChatModel()

## Create Evals, Register App

In [ ]:
from src.observability import create_evals
from trulens.providers.openai import OpenAI

provider = OpenAI()

evals = create_evals(provider = provider)

In [ ]:
from src.rag import Rag
from trulens.apps.app import TruApp

rag = Rag(chat_model=chat_model, vector_store=vector_store, use_context_filter=True)

tru_rag = TruApp(
    rag,
    app_name="RAG",
    app_version="snowflake-oss",
    feedbacks = evals
)

## Generate an Answer

Next, we define a query and use the retrieval function to find relevant documents.

In [ ]:
with tru_rag:
    llm_response = rag.retrieve_and_generate("What datasets are used to evaluate the RAG triad?")

## Display

In [ ]:
from IPython.display import HTML, display

html_content = f"<div style='white-space: pre-wrap; border: 1px solid #ccc; padding: 10px; background-color: white; color: black; font-size: 16px;'>{llm_response}</div>"
display(HTML(html_content))

## Run TruLens Dashboard

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard()